In [20]:
import librosa
from skimage import feature
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import soundfile as sf
import os
from pathlib import Path


***In progress**

In [17]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

# Load and initialize the BirdNET-Analyzer models.
analyzer = Analyzer()

Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.


In [18]:
output_folder='C:/Users/dgnhk/Woodcock-CNN/data/train_data/audio/1/'
wav_files = os.listdir(output_folder)

In [21]:
for file in wav_files:
    audio_path = Path(os.path.join(output_folder, file))
    recording = Recording(
        analyzer,
        audio_path,
        min_conf=0.0
    )
    recording.analyze()
    for detection in recording.detections:
        if detection['scientific_name']=='Scolopax rusticola':
            print(file, detection['confidence'])
    
    break

read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording Peenemuende_140525_327_4ch_1170.wav


<IPython.core.display.Javascript object>